### Minibatch Gradient Descent  
: 전체 데이터를 작은 양으로 균일하게 나눠 minibatch 하나씩 학습하는 방법  
=> 컴퓨터가 각 minibatch에 있는 데이터의 cost만 계산한 후에 gradient descent 수행
- 업데이트를 좀 더 빠르게 할 수 있다
- cost 계산 시 전체 데이터를 쓰지 않아서 잘못된 방향으로 학습할 수도 있다 (gradient descnet가 거칠게 줄어들 수 있음)

## PyTorch Dataset

In [2]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73,80,75],
                        [93,88,93],
                        [89,91,80],
                        [96,98,100],
                        [73,66,70]]
        self.y_data = [[152],[185],[180],[196],[142]]
        
    # 이 데이터셋의 총 데이터 수 반환    
    def __len__(self):
        return len(self.x_data)
    
    # 어떤 인덱스를 입력받았을 때 그에 상응하는 입출력 데이터 반환
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        
        return x, y
    
dataset = CustomDataset()

## PyTorch DataLoader
**batch_size = 2**  
- 각 minibatch의 크기
- 통상적으로 2의 제곱수로 설정  

**shuggle=True**  
- epoch마다 데이터셋을 섞어, 데이터가 학습되는 순서를 바꿈
=> 모델이 데이터셋의 순서를 외우는 것 방지 가능

In [3]:
from torch.utils.data import DataLoader

dataloader = DataLoader(
    dataset,
    batch_size = 2,
    shuffle = True,
)

## Full Code with Dataset and DataLoader
enumerate(dataloader) : minibatch 인덱스와 데이터를 받음  
len(datalodader) : 한 epoch당 minibatch 개수

In [9]:
import torch
import torch.nn.functional as F

import torch.nn as nn

class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1) # 입력차원 3, 출력차원 1
    
    # hypothesis 계산은 forward()에서. Gradient 계산은 backward()에서 알아서 해줌
    def forward(self, x): 
        return self.linear(x)

model = MultivariateLinearRegressionModel()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        
        prediction = model(x_train)
    
        cost = F.mse_loss(prediction, y_train)
    
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
        print('Epoch {:4d}/{} Batch: {}/{}, Cost: {:.6f}'.format(
            epoch,nb_epochs,batch_idx+1, len(dataloader), cost.item()
        ))

Epoch    0/20 Batch: 1/3, Cost: 14663.978516
Epoch    0/20 Batch: 2/3, Cost: 11277.992188
Epoch    0/20 Batch: 3/3, Cost: 2729.493164
Epoch    1/20 Batch: 1/3, Cost: 441.273621
Epoch    1/20 Batch: 2/3, Cost: 150.714554
Epoch    1/20 Batch: 3/3, Cost: 192.436447
Epoch    2/20 Batch: 1/3, Cost: 1.705238
Epoch    2/20 Batch: 2/3, Cost: 26.664827
Epoch    2/20 Batch: 3/3, Cost: 0.216577
Epoch    3/20 Batch: 1/3, Cost: 0.217702
Epoch    3/20 Batch: 2/3, Cost: 21.041777
Epoch    3/20 Batch: 3/3, Cost: 3.300607
Epoch    4/20 Batch: 1/3, Cost: 18.668602
Epoch    4/20 Batch: 2/3, Cost: 6.885032
Epoch    4/20 Batch: 3/3, Cost: 0.191099
Epoch    5/20 Batch: 1/3, Cost: 2.723767
Epoch    5/20 Batch: 2/3, Cost: 22.780983
Epoch    5/20 Batch: 3/3, Cost: 0.000135
Epoch    6/20 Batch: 1/3, Cost: 5.098173
Epoch    6/20 Batch: 2/3, Cost: 19.305616
Epoch    6/20 Batch: 3/3, Cost: 1.968023
Epoch    7/20 Batch: 1/3, Cost: 15.794948
Epoch    7/20 Batch: 2/3, Cost: 5.705020
Epoch    7/20 Batch: 3/3, Cost: 10